## 📦 Step 1: Install Dependencies

Auto-detects platform and installs required packages.

In [ ]:
import sys
import os

# Detect platform
def detect_platform():
    """Detect if running in Colab, Kaggle, or local environment."""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return 'kaggle'
    else:
        return 'local'

PLATFORM = detect_platform()
print(f"🖥️  Detected platform: {PLATFORM.upper()}")

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q flask flask-httpauth flask-limiter google-generativeai PyPDF2 python-docx langdetect cryptography requests

print("✅ Dependencies installed!")

## 🔧 Step 2: Setup Project

Clone repository (Colab/Kaggle) or use local installation (VSCode).

In [ ]:
if PLATFORM in ['colab', 'kaggle']:
    print("📥 Cloning repository...")
    if os.path.exists('expatLegalAdvisoryAgent'):
        !rm -rf expatLegalAdvisoryAgent
    !git clone -q https://github.com/KirthanaRupanagudi/expat-legal-advisory-agent.git expatLegalAdvisoryAgent
    
    # Add to Python path
    sys.path.insert(0, '/content/expatLegalAdvisoryAgent')
    print("✅ Repository cloned!")
else:
    # VSCode/Local: Assume running from project directory
    project_root = os.path.dirname(os.path.abspath('__file__'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    print("✅ Using local project files")

print(f"\n📂 Python path: {sys.path[0]}")

## 🔑 Step 3: Configure API Keys

**Required:** Google API Key for Gemini LLM and Google Translate

### Get your API key:
1. Go to [Google AI Studio](https://makersuite.google.com/app/apikey)
2. Create API key
3. Paste below

**Security:**
- Colab: Keys stored in session (lost on disconnect)
- VSCode: Set in `.env` file (not committed to git)

In [ ]:
from getpass import getpass
import os

def setup_api_keys():
    """Interactive API key setup with platform-specific helpers."""
    
    if PLATFORM == 'colab':
        print("🔐 Colab Secret Manager detected")
        try:
            from google.colab import userdata
            api_key = userdata.get('GOOGLE_API_KEY')
            if api_key:
                os.environ['GOOGLE_API_KEY'] = api_key
                print("✅ Loaded GOOGLE_API_KEY from Colab secrets")
                return
        except:
            pass
    
    # Check if already set
    if os.getenv('GOOGLE_API_KEY'):
        print("✅ GOOGLE_API_KEY already configured")
        return
    
    # Manual input
    print("\n🔑 Enter your Google API Key:")
    print("   Get it from: https://makersuite.google.com/app/apikey")
    api_key = getpass("API Key: ")
    
    if api_key:
        os.environ['GOOGLE_API_KEY'] = api_key
        print("✅ API key configured!")
    else:
        print("⚠️  No API key provided. Agent will not work.")

# Optional: Flask API key (only needed for REST API mode)
if not os.getenv('FLASK_API_KEY'):
    os.environ['FLASK_API_KEY'] = 'notebook-mode-key-12345'

setup_api_keys()

## 🚀 Step 4: Initialize Agent

Import and create the agent instance.

In [ ]:
from project.main_agent import run_agent
from project.tools.tools import extract_pdf_text, extract_docx_text

print("✅ Agent initialized successfully!")
print("\n📚 Available languages: English (en), Spanish (es), French (fr), Dutch (nl), German (de)")
print("\n💡 Tip: Use 'auto' for automatic document language detection")

## 📄 Step 5: Upload Documents (Optional)

Upload PDF or DOCX files for analysis.

In [ ]:
def upload_document():
    """Platform-aware file upload handler."""
    
    if PLATFORM == 'colab':
        from google.colab import files
        print("📤 Click 'Choose Files' to upload...")
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            return filename
    
    elif PLATFORM == 'kaggle':
        print("📂 Upload via Kaggle 'Add Data' or use input path:")
        filepath = input("Enter file path: ")
        return filepath if os.path.exists(filepath) else None
    
    else:  # VSCode/Local
        print("📂 Enter file path (or drag & drop file):")
        filepath = input("File path: ").strip().strip('"').strip("'")
        return filepath if os.path.exists(filepath) else None

def extract_document_text(filepath):
    """Extract text from PDF or DOCX file."""
    if not filepath or not os.path.exists(filepath):
        print("❌ File not found")
        return None
    
    try:
        if filepath.lower().endswith('.pdf'):
            print("📖 Extracting text from PDF...")
            text = extract_pdf_text(filepath)
        elif filepath.lower().endswith('.docx'):
            print("📖 Extracting text from DOCX...")
            text = extract_docx_text(filepath)
        else:
            print("❌ Unsupported file type. Use PDF or DOCX.")
            return None
        
        print(f"✅ Extracted {len(text)} characters")
        return text
    except Exception as e:
        print(f"❌ Error extracting text: {e}")
        return None

# Example usage
print("\n📄 To upload a document, run:")
print("   file_path = upload_document()")
print("   document_text = extract_document_text(file_path)")

## 💬 Step 6: Ask Questions

Interactive query interface with document support.

In [ ]:
def ask_agent(question, document_text=None, document_language='auto', preferred_language='en', verbose=True):
    """
    Ask the legal advisory agent a question.
    
    Args:
        question: Your question (in any language)
        document_text: Optional legal document content
        document_language: Language of document ('auto', 'en', 'es', 'fr', 'nl', 'de')
        preferred_language: Language for response ('en', 'es', 'fr', 'nl', 'de')
        verbose: Show processing details
    
    Returns:
        dict with 'response' and 'confidence' keys
    """
    if verbose:
        print("🤔 Processing your question...\n")
    
    try:
        result = run_agent(
            user_input=question,
            document_content=document_text,
            document_language=document_language,
            preferred_language=preferred_language
        )
        
        if verbose:
            print("="*60)
            print(f"📊 Confidence: {result.get('confidence', 0):.0%}")
            print("="*60)
            print(f"\n{result.get('response', 'No response generated')}")
            print("\n" + "="*60)
        
        return result
    
    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        print(error_msg)
        return {'response': error_msg, 'confidence': 0.0}

# Example usage
print("\n💡 Example queries:\n")
print("1. Simple question (no document):")
print("   ask_agent('What visa do I need to work in Germany?')\n")

print("2. With document analysis:")
print("   ask_agent('Am I eligible?', document_text=doc_text)\n")

print("3. Multi-language (question in Spanish, response in Spanish):")
print("   ask_agent('¿Qué visa necesito?', preferred_language='es')\n")

## 🎯 Quick Examples

Run these cells to test the agent.

In [ ]:
# Example 1: Simple question without document
ask_agent(
    question="What documents do I need to apply for a work visa in the Netherlands?",
    preferred_language='en'
)

In [ ]:
# Example 2: Question in Spanish, response in Spanish
ask_agent(
    question="¿Cuánto tiempo tarda el proceso de visa de trabajo?",
    preferred_language='es'
)

In [ ]:
# Example 3: With sample document text
sample_document = """
WORK PERMIT REQUIREMENTS - GERMANY

To apply for a work permit in Germany, you must:
1. Have a valid job offer from a German employer
2. Possess relevant qualifications (degree or vocational training)
3. Submit application at German embassy in home country
4. Provide proof of health insurance
5. Show financial stability (bank statements)

Processing time: 6-8 weeks
Fee: €75
"""

ask_agent(
    question="How long does the process take and what is the fee?",
    document_text=sample_document,
    document_language='en',
    preferred_language='en'
)

## 📤 Upload Your Own Document

Upload a PDF or DOCX file and ask questions about it.

In [ ]:
# Upload document
uploaded_file = upload_document()

if uploaded_file:
    # Extract text
    my_document = extract_document_text(uploaded_file)
    
    if my_document:
        print(f"\n✅ Document loaded! ({len(my_document)} characters)")
        print("\n📝 Preview (first 500 chars):")
        print(my_document[:500])
        print("\n" + "="*60)
        print("\n💬 Now ask questions about your document in the next cell!")
else:
    print("⚠️  No file uploaded. You can paste text directly instead.")

In [ ]:
# Ask questions about your uploaded document
# Replace 'my_document' with your variable name if different

if 'my_document' in locals():
    ask_agent(
        question="What are the main requirements mentioned in this document?",
        document_text=my_document,
        document_language='auto',  # Auto-detect
        preferred_language='en'
    )
else:
    print("⚠️  No document loaded. Upload one in the cell above first.")

## 🔄 Interactive Mode

Continuous conversation mode - ask multiple questions.

In [ ]:
def interactive_mode(document_text=None):
    """
    Interactive Q&A mode. Type 'quit' to exit.
    """
    print("\n" + "="*60)
    print("🎤 INTERACTIVE MODE")
    print("="*60)
    print("Type your questions (or 'quit' to exit)\n")
    
    # Get language preferences once
    print("Select response language (en/es/fr/nl/de) [default: en]: ", end="")
    lang = input().strip() or 'en'
    
    if document_text:
        print(f"\n✅ Using loaded document ({len(document_text)} chars)\n")
    
    while True:
        print("\n" + "-"*60)
        question = input("\n❓ Your question: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break
        
        if not question:
            continue
        
        ask_agent(
            question=question,
            document_text=document_text,
            preferred_language=lang
        )

# Start interactive mode
# Uncomment to use:
# interactive_mode(document_text=my_document if 'my_document' in locals() else None)

print("\n💡 To start interactive mode, uncomment the last line and run this cell")

## 🧪 Advanced: Batch Processing

Process multiple questions at once.

In [ ]:
def batch_process(questions_list, document_text=None, preferred_language='en'):
    """
    Process multiple questions and return results.
    
    Args:
        questions_list: List of question strings
        document_text: Optional document content
        preferred_language: Response language
    
    Returns:
        List of result dictionaries
    """
    results = []
    
    for i, question in enumerate(questions_list, 1):
        print(f"\n{'='*60}")
        print(f"Question {i}/{len(questions_list)}: {question}")
        print(f"{'='*60}")
        
        result = ask_agent(
            question=question,
            document_text=document_text,
            preferred_language=preferred_language,
            verbose=True
        )
        
        results.append({
            'question': question,
            'response': result.get('response'),
            'confidence': result.get('confidence')
        })
    
    return results

# Example batch processing
questions = [
    "What are the visa requirements?",
    "How long does the process take?",
    "What documents do I need?"
]

# Uncomment to run:
# batch_results = batch_process(questions, document_text=sample_document)

print("\n💡 Uncomment the last line to run batch processing")

## 📊 Export Results

Save conversation history to file.

In [ ]:
import json
from datetime import datetime

def save_results(results, filename=None):
    """
    Save Q&A results to JSON file.
    """
    if not filename:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'legal_advisory_results_{timestamp}.json'
    
    output = {
        'timestamp': datetime.now().isoformat(),
        'platform': PLATFORM,
        'results': results
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ Results saved to: {filename}")
    
    # Download in Colab
    if PLATFORM == 'colab':
        from google.colab import files
        files.download(filename)

# Example usage:
# save_results(batch_results)

print("\n💡 Use save_results(your_results) to export")

## 🆘 Troubleshooting

Common issues and solutions.

In [ ]:
def diagnose_issues():
    """
    Check configuration and diagnose common problems.
    """
    print("🔍 SYSTEM DIAGNOSTICS\n")
    print("="*60)
    
    # Platform
    print(f"Platform: {PLATFORM}")
    print(f"Python: {sys.version}")
    
    # API Keys
    print("\n🔑 API Keys:")
    google_key = os.getenv('GOOGLE_API_KEY')
    if google_key:
        print(f"  ✅ GOOGLE_API_KEY: Set ({google_key[:10]}...)")
    else:
        print("  ❌ GOOGLE_API_KEY: Not set")
        print("     → Run the API key setup cell above")
    
    # Dependencies
    print("\n📦 Dependencies:")
    required = [
        'flask', 'google.generativeai', 'PyPDF2', 
        'docx', 'langdetect', 'cryptography', 'requests'
    ]
    
    for pkg in required:
        try:
            __import__(pkg.replace('.', '_').replace('-', '_').split('.')[0])
            print(f"  ✅ {pkg}")
        except ImportError:
            print(f"  ❌ {pkg} - Run: !pip install {pkg}")
    
    # Project files
    print("\n📂 Project Structure:")
    try:
        from project.main_agent import run_agent
        print("  ✅ project.main_agent imported")
    except ImportError as e:
        print(f"  ❌ Cannot import project: {e}")
        print("     → Re-run setup cells above")
    
    print("\n" + "="*60)
    print("\n✅ If all checks pass, you're ready to use the agent!")

diagnose_issues()

---

## 📚 Additional Resources

- **Repository**: [GitHub](https://github.com/KirthanaRupanagudi/expat-legal-advisory-agent)
- **Supported Languages**: English, Spanish, French, Dutch, German
- **API Documentation**: [Google AI Studio](https://makersuite.google.com/)

## 💡 Tips

1. **For best results**: Provide clear, specific questions
2. **Document analysis**: Upload official legal documents for accurate advice
3. **Multi-language**: Ask in any language, get responses in your preferred language
4. **Privacy**: Documents are not stored permanently

## ⚖️ Disclaimer

This tool provides general legal information only. For specific legal advice, consult a qualified immigration attorney.